In [ ]:
import pandas as pd 
import numpy as np 
import torch

ModuleNotFoundError: No module named 'torch'

In [4]:
#Sample user reddit feed

data = {
    "Comment": [
        "The new policy will definitely impact elections…",
        "I think this is an overreaction by the media",
        "This subreddit is amazing for left-leaning discussions"
    ],
    "Subreddit": [
        "politics",
        "worldnews",
        "news"
    ],
    "Post Title": [
        "Election Policy Updates",
        "Global Media Coverage",
        "Community Reactions"
    ],
    "Post Type": [
        "hot",
        "top",
        "controversial"
    ],
    "Upvotes": [25, 15, 50],
    "Post Score": [100, 80, 200],
    "Comment Score": [10, 5, 20],
    "Post Date": ["2025-03-12", "2025-04-01", "2025-02-28"],
    "Comment Date": ["2025-03-12", "2025-04-01", "2025-02-28"]
}


sample_user_df = pd.DataFrame(data)

In [5]:
sample_user_df

,Comment,Subreddit,Post Title,Post Type,Upvotes,Post Score,Comment Score,Post Date,Comment Date
0,The new policy will definitely impact elections…,politics,Election Policy Updates,hot,25,100,10,2025-03-12,2025-03-12
1,I think this is an overreaction by the media,worldnews,Global Media Coverage,top,15,80,5,2025-04-01,2025-04-01
2,This subreddit is amazing for left-leaning dis...,news,Community Reactions,controversial,50,200,20,2025-02-28,2025-02-28


In [ ]:
def detect_bubble(user_feed_df, model, tokenizer, embedding_model, classifier_model, threshold=0.7):
    user_feed_df["text_combined"] = (
        user_feed_df["comment"].fillna("") + " " +
        user_feed_df["post_title"].fillna("") + " " +
        user_feed_df["post_body"].fillna("")
    )
    
    def get_embeddings(texts, batch_size=16):
        embeddings = []
        for i in range(0, len(texts), batch_size):
            batch = texts[i:i+batch_size].tolist()
            tokens = tokenizer(batch, padding=True, truncation=True, return_tensors="pt", max_length=256)
            with torch.no_grad():
                outputs = embedding_model(**tokens)
            batch_embeds = outputs.last_hidden_state[:, 0, :].numpy()
            embeddings.append(batch_embeds)
        return np.vstack(embeddings)
    
    text_embeddings = get_embeddings(user_feed_df["text_combined"])
    
    text_embeddings = get_embeddings(user_feed_df["text_combined"])
    
    
    engagement_features = ['post_score', 'post_upvotes', 'post_downvotes', 'comment_score',
                       'engagement_ratio', 'upvote_ratio']
    
    X_engagement = user_feed_df[engagement_features].values
    
    # Combine with engagement features
    X_feed = np.hstack([text_embeddings, user_feed_df[engagement_features].values])
    
    scores = model.predict(X_feed)
    
    avg_score = np.mean(scores)
    #what fraction of posts lean on one side, if enough posts on one side, in bubble
    prop_same_side = np.mean(scores > 0) if avg_score > 0 else np.mean(scores < 0)
    bubble = prop_same_side >= threshold